In [4]:
# general data science/utilization/visualization imports
import json
import os
import random

from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px

from sklearn.metrics import (accuracy_score, classification_report, f1_score,
                             precision_score, recall_score)
from sklearn.model_selection import train_test_split
# torch imports
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
# huggingface imports
import transformers
from datasets import load_metric
from transformers import (AutoModelForSequenceClassification, AutoTokenizer, 
                          Trainer, TrainingArguments)

# ray tune imports for hyperparameter optimization
from ray.tune.schedulers import ASHAScheduler, PopulationBasedTraining
from ray.tune.suggest.hyperopt import HyperOptSearch



In [5]:
def seed_all(seed):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

SEED=42
seed_all(SEED)

In [6]:
class TextClassificationDataset(Dataset):
    def __init__(self, dataframe):
        self.labels = dataframe.label.to_list()
        self.inputs = dataframe.content.to_list()
        self.labels_to_idx = {k:v for k,v in labels_dict.items()}

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        if type(idx)==torch.Tensor:
            idx = list(idx)

        input_data = self.inputs[idx]
        target = self.labels[idx]
        target = self.labels_to_idx[target]

        return {'text': input_data, 'label':target}

In [7]:
#@title Model training
model_name = 'cardiffnlp/twitter-roberta-base-sentiment'# @param {type:'string'}
# we will perform the search to optimize the model accuracy,
# we need to specify and load the accuracy metric as a first step
metric = load_metric("accuracy")


# since we already entered a model name, we can load the pretrained tokenizer
# we can also load the model but i'll describe it in the model_init function.
tokenizer = AutoTokenizer.from_pretrained(model_name)


def model_init():
    """
    Hyperparameter optimization is performed by newly initialized models,
    therefore we will need to initialize the model every single search run. 
    This function initializes and returns the pretrained model selected with `model_name`
    """
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5, return_dict=True, ignore_mismatched_sizes=True)

# we picked the metric to search for but haven't defined how to calculate it from predictions
# let's do that!
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1) # just pick the indices that has the maximum values
    return metric.compute(predictions=predictions, references=labels)

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [8]:
# @title Select the dataset file
file_name = "dataset-11.csv" # @param {type:"string"}
dataset = pd.read_csv(file_name)
print('Number of samples: %d'%len(dataset))
display(dataset.sample(5))

idx_to_label = dict(enumerate(dataset.label.unique()))
labels_dict = {v:k for k,v in idx_to_label.items()}
print(labels_dict)
px.pie(dataset, names=dataset.label.unique(), values=dataset.label.value_counts())

Number of samples: 346


,id,externalId,skipped,status,label,content
108,ckyd73h600h5r0l1t7irg2gmm,218,False,REVIEWED,INTERFACE,not able to scroll the code snippet inside the...
158,ckyd73qb903ef0lyq9sos8d3h,447,False,LABELED,INTERFACE,"latest update forces me to link twitter, and t..."
278,ckyd747hk0hns0l0uhsp4hrlw,895,False,LABELED,SUBSCRIPTION,have to pay to read news articles? no thanks.
209,ckyd73zbb0gqq0l0e7p2q6ze2,664,False,LABELED,SUBSCRIPTION,they don't give a single article without premi...
320,ckyd74gp20hcp0l164ens459v,1060,False,REVIEWED,SUBSCRIPTION,"what is this app, even two or four words are n..."


{'CONTENT': 0, 'USER_EXPERIENCE': 1, 'SUBSCRIPTION': 2, 'INTERFACE': 3}


In [9]:
# @title Automatic hyperparameter search with Ray backend.
# @markdown I've prepared common schedulers and algorithms to perform the search. Just pick the one you want from the dropdown menu.


# @markdown Note: You cannot use population based training with search algorithms so pick None as a search_algorithm if that's the case. 

schedulers = {
    'ASHA Scheduler': ASHAScheduler(metric='objective', mode='max'),
    'Population Based Training': PopulationBasedTraining(metric='objective', mode='max')
}

search_algos = {
    'None': None,
    'HyperOpt':HyperOptSearch(metric='objective', mode='max', random_state_seed=SEED),
}
scheduler = "ASHA Scheduler" # @param ["ASHA Scheduler", "Population Based Training"]
search_algorithm = "HyperOpt" # @param ["None", "HyperOpt"]
n_trials =  40# @param {type:"integer"}


scheduler = schedulers[scheduler]
search_algorithm = search_algos[search_algorithm]

In [10]:
# @title Utility Functions


# we will use this function to tokenize the input data
# I'm passing tokenizer as a default argument since 
# ray tune performs search in a black-box setting
# so passing what we need beforehand is vital and otherwise
# it'll throw an error.
def tokenize(sample, tokenizer=tokenizer):
    tokenized_sample = tokenizer(sample['text'], padding=True, truncation=True)
    tokenized_sample['label'] = sample['label']
    return tokenized_sample

def prepare_datasets(dataset_df, test_size=.2, val_size=.2):
    # split the dataset to training and test sets
    # then create a validation split from training set
    # it is generally advised to have stratified splits
    # this means that every split will have almost same ratio of different labels
    # otherwise, the model could overfit to dominant class or underfit the 
    # class with less samples.
    train_set, test_set = train_test_split(dataset_df, test_size=test_size,
                                        stratify=dataset_df.label, random_state=SEED)

    train_set, val_set = train_test_split(train_set, test_size=val_size,
                                        stratify=train_set.label, random_state=SEED)

    # shuffle the dataframes beforehand 
    train_set = train_set.sample(frac=1, random_state=SEED)
    val_set = val_set.sample(frac=1, random_state=SEED)
    test_set = test_set.sample(frac=1, random_state=SEED)

    # convert dataframes to torch datasets
    train_dataset = TextClassificationDataset(train_set)
    val_dataset = TextClassificationDataset(val_set)
    test_dataset = TextClassificationDataset(test_set)

    # tokenize the datasets
    tokenized_train_set = train_dataset.map(tokenize)
    tokenized_val_set = val_dataset.map(tokenize)
    tokenized_test_set = test_dataset.map(tokenize)

    return tokenized_train_set, tokenized_val_set, tokenized_test_set


def calculate_metrics(labels, preds, index):
    multi_indices = pd.MultiIndex.from_product([['Recall', 'Precision', 'F1'],['Weighted', 'Macro', 'Micro']])
    df = pd.DataFrame(columns=multi_indices, index=[index])

    df['Accuracy'] = accuracy_score(labels, preds)
    df[('Recall', 'Weighted')] = recall_score(labels, preds, average='weighted')
    df[('Recall', 'Macro')] = recall_score(labels, preds, average='macro')
    df[('Recall', 'Micro')] = recall_score(labels, preds, average='micro')


    df[('Precision', 'Weighted')] = precision_score(labels, preds, average='weighted') 
    df[('Precision', 'Macro')] = precision_score(labels, preds, average='macro')
    df[('Precision', 'Micro')] = precision_score(labels, preds, average='micro')


    df[('F1', 'Weighted')] = f1_score(labels, preds, average='weighted')
    df[('F1', 'Macro')] = f1_score(labels, preds, average='macro')
    df[('F1', 'Micro')] = f1_score(labels, preds, average='micro')

    return df

In [ ]:
tokenized_train_set, tokenized_val_set, tokenized_test_set = prepare_datasets(dataset)

print('Training set has %d samples'%len(tokenized_train_set))
print('Validation set has %d samples' %len(tokenized_val_set))
print('Test set has %d samples'%len(tokenized_test_set))

# let's define the training arguments, 
# skip_memory_metrics: Whether to skip adding of memory profiler reports to metrics.
#       passed True since it slows down the training and evaluation speed
# evaluation_strategy: evaluation strategy to adopt at training phase. 
#       "steps" means evaluation is done at every 500 steps by default 
#       (we can override this value by passing eval_steps parameter)
training_args = TrainingArguments(
    'trial_results',
    evaluation_strategy="steps",
    disable_tqdm=True,
    skip_memory_metrics=True,
)

trainer = Trainer(
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=tokenized_train_set,
    eval_dataset=tokenized_val_set,
    model_init=model_init,
    compute_metrics=compute_metrics
    )

best_run = trainer.hyperparameter_search(
                direction="maximize", 
                n_trials=n_trials,
                backend="ray",
                search_alg=search_algorithm,
                scheduler=scheduler
                )

Training set has 220 samples
Validation set has 56 samples
Test set has 70 samples


loading configuration file https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/7dd97280b5338fb674b5372829a05a1aaaa76f9f2fa71c36199f2ce1ee1104a0.4c7ca95b4fd82b8bbe94fde253f5f82e5a4eedefe6f86f6fa79efc903d6cfe60
Model config RobertaConfig {
  "_name_or_path": "cardiffnlp/twitter-roberta-base-sentiment",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-05,
  "max

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

storing https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/11065c9c045391e7a6b2bb2451b862074866aeddabaece3ef767540b48247a1c.a8b67614ee564f9fefd65a3a42566038ccf3e86668cb888d8ae05ec670696ba7
creating metadata file for /root/.cache/huggingface/transformers/11065c9c045391e7a6b2bb2451b862074866aeddabaece3ef767540b48247a1c.a8b67614ee564f9fefd65a3a42566038ccf3e86668cb888d8ae05ec670696ba7
loading weights file https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/11065c9c045391e7a6b2bb2451b862074866aeddabaece3ef767540b48247a1c.a8b67614ee564f9fefd65a3a42566038ccf3e86668cb888d8ae05ec670696ba7
All model checkpoint weights were used when initializing RobertaForSequenceClassification.

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base

== Status ==
Current time: 2022-01-22 13:56:53 (running for 00:00:00.24)
Memory usage on this node: 3.6/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: None
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/4.99 GiB heap, 0.0/2.49 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/_objective_2022-01-22_13-56-52
Number of trials: 1/40 (1 PENDING)
+---------------------+----------+-------+-----------------+--------------------+-------------------------------+---------+
| Trial name          | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_batch_size |    seed |
|---------------------+----------+-------+-----------------+--------------------+-------------------------------+---------|
| _objective_273bb9a8 | PENDING  |       |     3.86575e-05 |                  5 |                            64 | 5.38838 |
+---------------------+----------+-------+----------------

...

(ImplicitFunc pid=760) {'eval_loss': 1.0526759624481201, 'eval_accuracy': 0.7857142857142857, 'eval_runtime': 0.1664, 'eval_samples_per_second': 336.628, 'eval_steps_per_second': 42.079, 'epoch': 4.0}
== Status ==
Current time: 2022-01-22 14:00:08 (running for 00:03:15.60)
Memory usage on this node: 3.6/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: 0.6071428571428572
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/4.99 GiB heap, 0.0/2.49 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/_objective_2022-01-22_13-56-52
Number of trials: 13/40 (1 PENDING, 1 RUNNING, 11 TERMINATED)
+---------------------+------------+----------------+-----------------+--------------------+-------------------------------+----------+-------------+
| Trial name          | status     | loc            |   learning_rate |   num_train_epochs |   per_device_train_batch_size |     seed |   objective |
|-------

(ImplicitFunc pid=801) Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized because the shapes did not match:
(ImplicitFunc pid=801) - classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
(ImplicitFunc pid=801) - classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([5]) in the model instantiated
(ImplicitFunc pid=801) You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
(ImplicitFunc pid=801) Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


== Status ==
Current time: 2022-01-22 14:00:19 (running for 00:03:26.37)
Memory usage on this node: 5.4/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: 0.6071428571428572
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/4.99 GiB heap, 0.0/2.49 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/_objective_2022-01-22_13-56-52
Number of trials: 13/40 (1 PENDING, 1 RUNNING, 11 TERMINATED)
+---------------------+------------+----------------+-----------------+--------------------+-------------------------------+----------+-------------+
| Trial name          | status     | loc            |   learning_rate |   num_train_epochs |   per_device_train_batch_size |     seed |   objective |
|---------------------+------------+----------------+-----------------+--------------------+-------------------------------+----------+-------------|
| _objective_8f269f2e | RUNNING    | 172.28.0.2:801 |     2

(ImplicitFunc pid=801) Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


(ImplicitFunc pid=801) {'train_runtime': 10.4707, 'train_samples_per_second': 105.055, 'train_steps_per_second': 3.343, 'train_loss': 1.041999053955078, 'epoch': 5.0}
Result for _objective_8f269f2e:
  date: 2022-01-22_14-00-29
  done: true
  epoch: 5.0
  eval_accuracy: 0.6428571428571429
  eval_loss: 0.9779671430587769
  eval_runtime: 0.1678
  eval_samples_per_second: 333.659
  eval_steps_per_second: 41.707
  experiment_id: 4e9b5d4d7cfe420488c9e0ffbcd1ae64
  hostname: cd01fd1e6201
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  objective: 0.6428571428571429
  pid: 801
  time_since_restore: 17.509552001953125
  time_this_iter_s: 17.509552001953125
  time_total_s: 17.509552001953125
  timestamp: 1642860029
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 8f269f2e
  
(ImplicitFunc pid=801) {'eval_loss': 0.9779671430587769, 'eval_accuracy': 0.6428571428571429, 'eval_runtime': 0.1678, 'eval_samples_per_second': 333.659, 'eval_steps_per_second': 41.707, 'epoch': 5.0}
==

In [ ]:
best_run_params = best_run.hyperparameters
for param, val in best_run_params.items():
    setattr(trainer.args, param, val)
setattr(trainer.args, 'seed', int(np.floor(best_run_params['seed'])))

trainer.train()

In [ ]:
outputs = trainer.predict(tokenized_test_set)
logits, labels, metrics = outputs.predictions, outputs.label_ids, outputs.metrics
preds = np.argmax(logits, axis=-1)
results = calculate_metrics(labels, preds, file_name)
#results.to_csv('result_%s'%file_name)
results